# Testing tails for fair generation
Tests to ensures that high SNR events are fairly sampled:
- Low-$z$
- High $(m_1, m_2)$

In [ ]:
import numpy as np
from scipy import interpolate, optimize, integrate
import matplotlib.pyplot as plt

import extrapops.constants as const

## Low-$z$
Check that when generating with max high-$z$ of 1, we generate fairly between $z$-min and some small z after which fair generation is obvious.

In [ ]:
# Quick inspection of the potentially-problematic region -- z_min to 0.02

from extrapops.redshifts import _default_z_perdecade, sample_z, event_rate_persec

z_min, z_max = 1e-5, 1

test_redshift_params = {
    "z_range": [z_min, z_max],
    "T_yr": 10000,
    "merger_rate_model": "madau",
    "merger_rate_params": {
        "R_0": 18, "z_0": 0, "d": 2.7, "r": -2.9, "z_peak": 1.86}
}

z_sample = sample_z(**test_redshift_params)

z_max_test = 0.02

low_z_sample = z_sample[np.where(z_sample < z_max_test)[0]]
del z_sample

In [ ]:
low_zs = np.logspace(np.log10(z_min), np.log10(z_max_test), 500)
pdf_z = lambda z: test_redshift_params["T_yr"] * const.yr_s * event_rate_persec(z)

plt.figure(figsize=(10, 5), facecolor="1")
plt.title("Zoom low z (noisier events)")
plt.hist(low_z_sample, density=True, bins=100, histtype="step")
plt.plot(low_zs, pdf_z(low_zs) / len(low_z_sample))
plt.loglog()
plt.show()
plt.close()

In [ ]:
# Strange result, but see tests below:

In [ ]:
from extrapops.redshifts import _invCDF_interpolator

# Find x mapped to right bound
func_to_minimize = lambda x: np.abs(interpolate.splev(x, _invCDF_interpolator) - z_max_test)
sol = optimize.minimize(func_to_minimize, 1e-5, method='L-BFGS-B')
x_of_z_max_test = sol["x"][0]
z_approx = interpolate.splev(x_of_z_max_test, _invCDF_interpolator)
assert np.abs(z_approx - z_max_test) < 1e-3
print(f"{x_of_z_max_test=} maps to {z_approx}")

In [ ]:
# TEST #1: behaviour of numpy.random.random below this x_of_z_max_test,
#          when sampling over the whole interval

n_samples_uniform = int(5000 / x_of_z_max_test)
samples_uniform = np.random.random(n_samples_uniform)
low_x_samples = samples_uniform[np.where(samples_uniform < x_of_z_max_test)[0]]
del samples_uniform

print("This should be a uniform distribution!")
plt.figure(figsize=(10, 5), facecolor="1")
plt.hist(low_x_samples)
plt.show()
plt.close()

In [ ]:
# TEST #2: the interpolator works as expected in the interval z_min to z_max_test

xs = np.logspace(-18, np.log10(x_of_z_max_test), 100)
zs = interpolate.splev(xs, _invCDF_interpolator)

# recompute CDF at a few points with a different algorithm
z_subsample = np.logspace(np.log10(z_min), np.log10(z_max_test), 5000)
# add the last point to normalize the cdf
z_subsample = list(z_subsample) + [z_max]
dx_integral = 1e-7
ints = [(lambda zs: integrate.simps(pdf_z(zs), zs))(np.arange(z_min, z_int, dx_integral))
        for z_int in z_subsample[1:]]
ints = np.array([0] + list(ints))
CDF_samples = (ints - min(ints)) / (max(ints) - min(ints))

In [ ]:
print("Both sets should fall on top of each other!")
plt.figure(figsize=(10, 5), facecolor="1")
plt.plot(xs, zs, ".-", label="Interpolator")
plt.plot(CDF_samples[:-1], z_subsample[:-1], "o", label="Recomputed integral")
plt.loglog()
plt.ylabel("Redshift")
plt.legend()
plt.show()
plt.close()

In [ ]:
# Looks ok!
# Notice from the initial inspection that generating events with z<1e-3 is very unlikely,
# so this last test should be enough!

# We are generating in the low-z tail correctly!!!

## High $(m_1, m_2)$
### (Can be run independently from 1st part, except 1st import cell)
We'll test the methods used by default: invCDF for $m_1$, and rejection sampling for $m_2$.

In [ ]:
from extrapops.mass import sample_m1_m2, pdf_m1, pdf_m1_m2, _default_mass_params
from extrapops.redshifts import avg_n_events

# Let's use the default parameters. Where exactly m_max is should not matter too much.

m_min_test = 80
assert m_min_test < _default_mass_params["m_range"][1], \
    f"Need m_min_test smaller than upper mass bound {_default_mass_params['m_range'][1]}!"

n_samples = int(avg_n_events() / 1)
m1, m2 = sample_m1_m2(n_samples)

In [ ]:
# m1

m_log10range = np.log10(_default_mass_params["m_range"])
m1s = np.logspace(np.log10(m_min_test), np.log10(_default_mass_params["m_range"][1]), 10000)

nbins = 1000
plt.figure(figsize=(15, 5))
plt.plot(m1s, pdf_m1(m1s), label=r"$P(m_1)$")
plt.hist(m1, density=True, bins=nbins, histtype="step",
         label="sample")
plt.legend()
plt.title("m1 sample")
plt.xlim(m_min_test, _default_mass_params["m_range"][1] * 1.02)
plt.loglog()
plt.show(block=False)

In [ ]:
# Looks OK for m1!

In [ ]:
# m2

# we first need p(m2)
print("Marginalising over m1 to get pdf(m2)...")
# To avoid hitting 0's in the probability, start from m_min + eps
m_min_int = _default_mass_params["m_range"][0] + 1e-10
# To avoid integrating a long null tail, cut the upper bound
m_max_int = 0.9999 * _default_mass_params["m_range"][1]
ms_marg = np.logspace(np.log10(m_min_test), np.log10(m_max_int), 50, base=10)
pdf_m2_marg_m1 = np.array(
    [integrate.quad(lambda m1: pdf_m1_m2(m1, m2), m_min_int, m_max_int, limit=1000)[0]
     for m2 in ms_marg])

In [ ]:
nbins = 500
plt.figure(figsize=(15, 5))
plt.plot(ms_marg, pdf_m2_marg_m1, ".-", label=r"$P(m_2)$")
plt.hist(m2, density=True, bins=nbins, histtype="step", label="sample")
plt.legend()
plt.title("m2 sample")
plt.xlim(m_min_test, _default_mass_params["m_range"][1] * 1.02)
plt.loglog()
plt.show(block=False)

In [ ]:
# OK for m2 too! (the "decay" in the pdf is bc of the upper integration limit for the marginalisation)

# Probably the problematic boundary here is m_min, which does not matter too much,
# either for observable events or the background.